In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster 
KMeans
import re
import json
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import pandas as pd 
from IPython.display import display
from transformers  import pipeline
import torch

c:\nlp\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_reviews(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        reviews = file.readlines()
    return reviews



In [3]:
# Charger les titres des films depuis le fichier JSON
def load_movie_titles(json_file):
    with open(json_file, 'r', encoding='utf-8') as file:
        movie_titles = json.load(file)
    return movie_titles



In [4]:
# Fonction pour compter les critiques en ignorant les espaces et les titres
def count_reviews(reviews, movie_titles):
    total_reviews = 0
    
    # Parcourir chaque ligne du fichier de critiques
    for line in reviews:
        line = line.strip()  # Enlever les espaces en début et fin de ligne

        # Si la ligne est vide, on l'ignore
        if not line:
            continue

        # Si la ligne commence par "Film : ", on extrait le titre du film
        if line.startswith("Film : "):
            title = line[len("Film : "):].strip()  # Extraire le titre du film
            if title in movie_titles:
                continue  # Ignorer cette ligne car c'est un titre de film
        
        # Si la ligne est une critique (autre que titre et espace), on l'ajoute au total
        total_reviews += 1

    return total_reviews

In [5]:
# Charger les fichiers
file_path = 'critiques_films.txt'  # Chemin vers le fichier de critiques
json_file = r'C:\nlp\movie_titles.json'  # Chemin vers le fichier JSON des titres

reviews = load_reviews(file_path)
movie_titles = load_movie_titles(json_file)

# Compter le nombre de critiques
total_reviews = count_reviews(reviews, movie_titles)

# Afficher le nombre total de critiques
print(f"Nombre total de critiques : {total_reviews}")

Nombre total de critiques : 750


In [6]:
def preprocess_reviews(reviews, movie_titles):
    processed_reviews = []
    
    for review in reviews:
        review = review.strip()  # Enlever les espaces au début et à la fin de la ligne

        # Si la ligne est vide ou si la ligne est un titre de film, on l'ignore
        if not review or review.startswith("Film : "):
            continue

        # Sinon, on prétraite la critique
        review = review.lower()  # Mettre en minuscules
        review = re.sub(r'\d+', '', review)  # Supprimer les chiffres
        review = re.sub(r'[^\w\s]', '', review)  # Supprimer la ponctuation
        processed_reviews.append(review)
    
    return processed_reviews

# Appliquer le prétraitement
reviews_cleaned = preprocess_reviews(reviews, movie_titles)
print(f"Exemple de critique nettoyée : {reviews_cleaned[16]}")
print(f"Nombre de critiques nettoyées : {len(reviews_cleaned)}")

Exemple de critique nettoyée : on aurait pu sattendre de le part de spielberg à un manichéisme bien politiquement correct on a au contraire une oeuvre sobre dotée dun regard juste et réfléchi sur la nature humaine le réalisateur a réussi à éviter tous les écueils dans lesquels il aurait pourtant été facile de tomber  cest dautant plus admirable emouvant parce que brillant
Nombre de critiques nettoyées : 750


In [7]:
with open('movie_titles.json', 'r') as file:
    movie_titles = json.load(file)



# Créer un DataFrame avec les critiques
df = pd.DataFrame({'critique': reviews_cleaned})

# Assigner un titre de film à chaque critique. Répartir les 50 titres parmi les 750 critiques.
df['film_title'] = [movie_titles[i // (len(reviews_cleaned) // 50)] for i in range(len(reviews_cleaned))]

# Vérifier la structure du DataFrame
display(df.sample(10))

,critique,film_title
557,film culte et grand classique du cinéma ce éni...,Les Temps modernes
436,jhabite a tokyo depuis ans et jai pu voir kim...,Your Name
456,coco pourrait très bien marquer un tournant da...,Coco
193,surement le meilleur western mis en scène serg...,"Le Bon, la brute et le truand"
394,je crois que jen attendais un peu trop et que ...,Parasite
459,cest un magnifique voyage que nous propose cet...,Coco
679,il y a très longtemps dans une galaxie lointa...,Star Wars : Episode IV - Un nouvel espoir (La ...
117,avec césars à son actif et considéré comme lu...,Vol au-dessus d'un nid de coucou
258,premier volet dune des plus grandes trilogies ...,Le Seigneur des anneaux : la communautÃ© de l'...
125,sil ne fallait retenir quun seul disney ce ser...,Le Roi Lion


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.nn.functional import softmax

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Ensure to move the model to the correct device (GPU/CPU)
model = model.to('cuda') if torch.cuda.is_available() else model.to('cpu')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Définir le device (GPU si disponible, sinon CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assurez-vous que le modèle est aussi sur le bon device
model = model.to(device)

def split_review(review, max_length=512):
    # Tokenisation de la critique
    tokens = tokenizer.tokenize(review)
    
    # Diviser les tokens en morceaux de taille 'max_length'
    chunks = [tokens[i:i+max_length] for i in range(0, len(tokens), max_length)]
    
    # Recomposer les morceaux en texte
    return [tokenizer.convert_tokens_to_string(chunk) for chunk in chunks]

In [10]:
def get_sentiment_for_review(review):
    # Diviser la critique en morceaux
    chunks = split_review(review)

    sentiments = []

    # Analyser chaque morceau
    for chunk in chunks:
        # Tokenisation et envoi sur le device (GPU/CPU)
        inputs = tokenizer(chunk, truncation=True, padding=True, max_length=512, return_tensors='pt').to(device)
        outputs = model(**inputs)  # Obtenir les logits du modèle
        logits = outputs.logits

        # Appliquer softmax pour obtenir les probabilités
        probs = softmax(logits, dim=1)

        # Ajouter la probabilité de la classe dominante
        sentiments.append(probs.max().item())  # Probabilité de la classe la plus probable

    # Calculer la moyenne des sentiments
    final_sentiment = sum(sentiments) / len(sentiments)
    
    return final_sentiment

In [11]:
df['sentiment'] = df['critique'].apply(get_sentiment_for_review)



In [12]:
# Voir le DataFrame avec les sentiments
print(df[['film_title', 'critique', 'sentiment']].sample(10))

                                            film_title  \
9                                         Forrest Gump   
80                                         Les EvadÃ©s   
675  Star Wars : Episode IV - Un nouvel espoir (La ...   
692               Spider-Man : Across The Spider-Verse   
6                                         Forrest Gump   
147                 The Dark Knight, Le Chevalier Noir   
613                            Le Tombeau des lucioles   
710                               Le Voyage de Chihiro   
212                                          Gladiator   
100         Le Seigneur des anneaux : le retour du roi   

                                              critique  sentiment  
9    forrest gump est sans aucun doute le meilleur ...   0.507717  
80   des films sur lunivers carcéral il y a en a eu...   0.519991  
675  ah ce film immortel magnifique novateur george...   0.508543  
692  je lattendais depuis longtemps et cest une vra...   0.520420  
6    la vie cest comm

In [15]:
df_sorted = df.sort_values(by='sentiment', ascending=True)

# Afficher les 10 lignes ayant les moindres sentiments
print(df_sorted[['film_title', 'critique', 'sentiment']].head(10))

               film_title                                           critique  \
443             Your Name  je ne suis pas amateur de ce genre de film jai...   
451                  Coco  que dire du nouveau disneypixar  en un mot  su...   
68             Le Parrain  francis ford coppola signait en  un véritable ...   
44         La Ligne verte  un superbe film fantastique et émouvant dans l...   
66             Le Parrain  le parrain est un film de gangsters culte et l...   
652              Psychose  vèritable film phare dalfred hitchcock ou susp...   
480  Retour vers le futur  attachez vos ceintures grisant et diablement s...   
319           Gran Torino  écrire sa critique sur gran torino en  cest ma...   
659              Psychose  une référence du thriller signé dune main de m...   
465             Inception  après le chef doeuvre superhéroïque the dark k...   

     sentiment  
443   0.500069  
451   0.500151  
68    0.500152  
44    0.500172  
66    0.500217  
652   0.500223  


In [16]:
from transformers import BartForConditionalGeneration, BartTokenizer


model_name = "facebook/bart-large-cnn"  # Modèle pré-entraîné pour le résumé
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [17]:
def split_text_into_batches(text, max_length=1024):
    tokens = tokenizer.tokenize(text)
    chunks = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]
    return [tokenizer.convert_tokens_to_string(chunk) for chunk in chunks]


In [18]:
def generate_summary_for_long_text(text, max_length=130, min_length=30):
    chunks = split_text_into_batches(text)
    summaries = []
    
    for chunk in chunks:
        inputs = tokenizer(chunk, max_length=1024, truncation=True, return_tensors="pt")
        
        # Si vous avez un GPU
        if torch.cuda.is_available():
            model.to("cuda")
            inputs = {key: value.to("cuda") for key, value in inputs.items()}
        
        summary_ids = model.generate(
            inputs["input_ids"], 
            max_length=max_length, 
            min_length=min_length, 
            length_penalty=2.0, 
            num_beams=4, 
            early_stopping=True
        )
        summaries.append(tokenizer.decode(summary_ids[0], skip_special_tokens=True))
    
    # Combinez les résumés des morceaux
    return " ".join(summaries)


In [19]:
# Groupement des critiques par film
grouped_reviews = df.groupby('film_title')['critique'].apply(lambda x: ' '.join(x)).reset_index()

# Générer un résumé pour chaque film
grouped_reviews['summary'] = grouped_reviews['critique'].apply(generate_summary_for_long_text)


In [20]:
print(grouped_reviews[['film_title', 'summary']].head())


                film_title                                            summary
0     12 hommes en colÃ¨re  Hommes en colère fait partie de cette catégori...
1       American History X  derek et danny sont deux frères en plein cœur ...
2                     Coco  en  après le rachat tout frais de pixar par di...
3         Django Unchained  django unchained est avant tout un grand spect...
4  Dune : DeuxiÃ¨me Partie  que dire chaque plan est éblouissant cest spec...


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
import heapq


In [37]:
french_stop_words = [
    "le", "la", "les", "de", "des", "en", "un", "une", "et", "à", "dans", "du", "que", "qui", "ce", "il", "elle", "nous", "vous", 
    "ils", "elles", "par", "pour", "avec", "sur", "se", "ne", "pas", "plus", "tout", "tous", "toute", "toutes", "au", "aux", "sont",
    "a", "au", "une", "l", "m", "n", "y", "le", "la", "les"
]

In [38]:
def extract_keywords(text, n=5):
    # Utilisation de TfidfVectorizer pour extraire les mots-clés en français
    tfidf_vectorizer = TfidfVectorizer(stop_words=french_stop_words)
    tfidf_matrix = tfidf_vectorizer.fit_transform([text])
    feature_names = tfidf_vectorizer.get_feature_names_out()
    
    # Calcul des scores des mots-clés
    scores = tfidf_matrix.sum(axis=0).A1
    word_score = {word: scores[idx] for idx, word in enumerate(feature_names)}
    
    # Récupération des n mots-clés les plus importants
    top_keywords = heapq.nlargest(n, word_score, key=word_score.get)
    return top_keywords

In [39]:
def generate_summary(critique, keywords, max_lines=4):
    # Diviser la critique en phrases
    sentences = critique.split('. ')
    
    # Filtrer les phrases qui contiennent les mots-clés
    relevant_sentences = [sentence for sentence in sentences if any(keyword in sentence for keyword in keywords)]
    
    # Prendre jusqu'à `max_lines` phrases pertinentes
    summary = '. '.join(relevant_sentences[:max_lines]) + '.'
    return summary

In [42]:
summaries = []
for index, row in grouped_reviews.iterrows():
    # Extraire les mots-clés
    keywords = extract_keywords(row['summary'])
    
    # Générer un résumé condensé
    summary = generate_summary(row['summary'], keywords)
    summaries.append(summary)

# Ajouter les résumés au DataFrame
grouped_reviews['condensed_summary'] = summaries

# Afficher les résultats
#print(grouped_reviews[['film_title', 'condensed_summary']])

print(grouped_reviews["summary"][0])
print(grouped_reviews["condensed_summary"][0])
print(len(grouped_reviews["summary"][0]))
print(len(grouped_reviews["condensed_summary"][0]))

Hommes en colère fait partie de cette catégorie de films. Les techniques cinématographiques qui changent ne perdent pas de leur impact initial. Les jurés du film des jurés aux personnalités remarquablement étudiées. ide de culpabilité quils ont prononcé contre un jeune homme accusé de meurtre la mise en scène théâtrale est un véritable atout latmosphère claustrophobe du film est prenante. Acteurs tirent parfaitement son épingle de jeu that ce soit lee j cobb ou bien évidemment henry fonda une fois de plus grandiose. Ce juré nest pas totalement convaincu non plus de sa culpabilité chacune des pièces de laccusation  un couteau des témoignages le plan dun appartement etc. va donc être passé au crible par le jury dans le désordre des souvenirs de chacun. Le film au départ multiplie les plans densemble la caméra. re lindividualité de chacun des personnages en les plaçant toujours adroitement dans le cadre chaque plan ninclut en effet. des classes sociales et des origines différentes sidney 

In [44]:
grouped_reviews.to_csv('grouped_reviews.csv', index=False)

In [58]:
s=0
for i in range(14):
    s = len(df["critique"][i]) + s

print(s)
print("les critiques du premier film par exemple contiennent ", s, "mots")

20773
les critiques du premier film par exemple contiennent  20773 mots


In [57]:
#reduction du nombre des mots 
print(len(grouped_reviews["summary"][1]))
print(len(grouped_reviews["condensed_summary"][1]))


1221
495
